In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
# Question 3

# 3-a
cols=['Number of times pregnant', 
      'Plasma glucose concentration a 2 hours in an oral glucose tolerance test',
      'Diastolic blood pressure (mm Hg)', 
      'Triceps skinfold tickness (mm)', 
      '2-Houre serum insulin (mu U/ml)',
      'Body mass index (weight in kg/(height in m)^2)', 
      'Diabetes pedigree function', 
      'Age (years)', 
      'Class variable (0 or 1)'] 

df = pd.read_csv('Diabetes.csv', names=cols, header=None)
display(df)

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skinfold tickness (mm),2-Houre serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [2]:
# find the number of null values and missing values in each column
for col in df.columns:
    x=pd.to_numeric(df[col], errors='coerce')
    idx = x.isna()
    seen = set()
    uniq = [x for x in df[idx][col] if x not in seen and not seen.add(x)]
    for j in df[idx][col]:
        df[col] = df[col].replace({j : np.NaN})
        
df.isna().sum()

Number of times pregnant                                                    0
Plasma glucose concentration a 2 hours in an oral glucose tolerance test    0
Diastolic blood pressure (mm Hg)                                            0
Triceps skinfold tickness (mm)                                              0
2-Houre serum insulin (mu U/ml)                                             0
Body mass index (weight in kg/(height in m)^2)                              0
Diabetes pedigree function                                                  0
Age (years)                                                                 0
Class variable (0 or 1)                                                     0
dtype: int64

In [3]:
# removing outlier records from dataset using z-score method
correct_record = []
df_zscore = df.copy()
for col in df_zscore.columns:
    if col != "Class variable (0 or 1)":
        df_zscore['_z'] = stats.zscore(df_zscore[col])
        qu = '_z' + ' <= 3 & ' + '_z' + ' >= -3'
        Without_Outliers = df_zscore.query(qu)
        df_zscore =  Without_Outliers.copy()
        df_zscore[col] = df_zscore['_z']
    
df_zscore.drop(['_z'], axis=1, inplace=True)
print('Outliers record number using z_score \t', df.shape[0] - df_zscore.shape[0])
display(df_zscore)

Outliers record number using z_score 	 83


,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skinfold tickness (mm),2-Houre serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable (0 or 1)
0,0.639947,0.852734,0.148818,0.868668,-0.719459,0.200687,0.510998,1.434609,1
1,-0.844885,-1.120621,-0.159937,0.487391,-0.719459,-0.760858,-0.370909,-0.190593,0
2,1.233880,1.949043,-0.262856,-1.355445,-0.719459,-1.214158,0.654787,-0.105056,1
3,-0.844885,-0.995328,-0.159937,0.106115,0.084023,-0.554812,-0.958847,-1.045963,0
5,0.342981,-0.149605,0.251737,-1.355445,-0.719459,-0.898221,-0.850206,-0.276130,0
...,...,...,...,...,...,...,...,...,...
763,1.827813,-0.619451,0.354655,1.694767,0.819123,0.104533,-0.946065,2.546589,0
764,-0.547919,0.038334,0.045900,0.360299,-0.719459,0.640251,-0.406057,-0.532741,0
765,0.342981,0.007011,0.148818,0.106115,0.237881,-0.815803,-0.709612,-0.276130,0
766,-0.844885,0.163626,-0.468693,-1.355445,-0.719459,-0.280085,-0.377299,1.177998,1


In [4]:
y = df_zscore[['Class variable (0 or 1)']]
x = df_zscore.drop(['Class variable (0 or 1)'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 5)

In [5]:
rfy = np.ravel(y_train)
rf01 = RandomForestClassifier(n_estimators = 100, criterion = "entropy", max_depth = 13).fit(x_train, rfy)

In [ ]:
# 3_b
y_pred = rf01.predict(x_test)

In [6]:
# 3_c
print("=====> model result without removing outliers <=====")
c_mat = confusion_matrix(y_test, y_pred)
display(c_mat)

=====> model result without removing outliers <=====


array([[76, 16],
       [13, 32]], dtype=int64)

In [7]:
TN = 0; TP = 0; FN = 0; FP = 0
d = y_test.to_numpy()
row_num = y_test.shape[0]

for i in range(0, row_num):
    if (d[i][0] == y_pred[i]) and (y_pred[i] == 0):
        TN += 1
    elif (d[i][0] == y_pred[i]) and (y_pred[i] == 1):
        TP += 1
    elif (d[i][0] != y_pred[i]) and (y_pred[i] == 0):
        FN += 1
    elif (d[i][0] != y_pred[i]) and (y_pred[i] == 1):
        FP += 1

print("TN =",TN, "\tTP =", TP,"\tFN =", FN,"\tFP =", FP)

TN = 76 	TP = 32 	FN = 13 	FP = 16


In [8]:
# 3_d
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84        92
           1       0.67      0.71      0.69        45

    accuracy                           0.79       137
   macro avg       0.76      0.77      0.76       137
weighted avg       0.79      0.79      0.79       137

